In [116]:
import xgboost
import lightgbm
import missingno
import sklearn

print(xgboost.__version__)
print(lightgbm.__version__)
print(missingno.__version__)
print(sklearn.__version__)

1.4.2
3.3.0
0.5.0
1.0


##  0. 라이브러리 impoort

In [117]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

## 1. 데이터로드

In [118]:
train_data_path = join('~/aiffel/kaggle_kakr_housing/data', 'train.csv')
sub_data_path = join('~/aiffel/kaggle_kakr_housing/data', 'test.csv')

x_train = pd.read_csv(train_data_path)
x_test = pd.read_csv(sub_data_path)

y_train = x_train['price']

del x_train['price']  

In [119]:
train_len = len(x_train)
train_len

15035

In [120]:
x_train.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


## 2. 데이터 전처리

In [122]:
## 전처리를 위해 일단 합친다

## train -test 합치기
data = pd.concat((x_train, x_test), axis=0)
sub_id = data['id'][train_len:]

# 결측치 삭제 : 없음
for c in data.columns:
    print('{} : {}'.format(c, len(data.loc[pd.isnull(data[c]), c].values)))

# id, date 불필요한 변수 삭제

del data['id']
data['date'] = data['date'].apply(lambda x : str(x[:6])).astype(str)



# 정규분포로 맞춤.
skew_columns = ['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_lot15', 'sqft_living15']

for c in skew_columns:
    data[c] = np.log1p(data[c].values)
    

## 다시 train test 나누기
    
x_test = data.iloc[train_len:, :]
x_train = data.iloc[:train_len, :]



print(x_train.shape)
print(x_test.shape)


id : 0
date : 0
bedrooms : 0
bathrooms : 0
sqft_living : 0
sqft_lot : 0
floors : 0
waterfront : 0
view : 0
condition : 0
grade : 0
sqft_above : 0
sqft_basement : 0
yr_built : 0
yr_renovated : 0
zipcode : 0
lat : 0
long : 0
sqft_living15 : 0
sqft_lot15 : 0
(15035, 19)
(6468, 19)


## 3.모델생성 및 학습 및 예측

#### 1. 앙상블 ( 앙상블에 부스팅회귀 모델 2개 추가해서 예측해봄)


In [123]:


gboost = GradientBoostingRegressor(random_state=2019)
xgboost = xgb.XGBRegressor(random_state=2019)
lightgbm = lgb.LGBMRegressor(random_state=2019)

## 두개 더추가 

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet

rfr = RandomForestRegressor(random_state=2019)
elnet = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=2019)



models = [{'model':gboost, 'name':'GradientBoosting'}, {'model':xgboost, 'name':'XGBoost'},
          {'model':lightgbm, 'name':'LightGBM'}, {'model':rfr, 'name':'rfr'},
          {'model':elnet, 'name':'elnet'}]

In [124]:
#from ngboost import NGBRegressor


In [125]:
# 성능평가
def get_cv_score(models):
    kfold = KFold(n_splits=5).get_n_splits(x_train.values)
    for m in models:
        CV_score = np.mean(cross_val_score(m['model'], X=x_train.values, y=y_train, cv=kfold))
        print(f"Model: {m['name']}, CV score:{CV_score:.4f}")


get_cv_score(models)

Model: GradientBoosting, CV score:0.8598
Model: XGBoost, CV score:0.8860
Model: LightGBM, CV score:0.8819
Model: rfr, CV score:0.8696
Model: elnet, CV score:0.6233


In [126]:
## 데이터 값 다시 되돌려서 rmse 확인하기

# 서브미션 파일 생성
def AveragingBlending(models, x, y, x_test):
    for m in models : 
        m['model'].fit(x.values, y)
    
    predictions = np.column_stack([
        m['model'].predict(x_test.values) for m in models
    ])
    return np.mean(predictions, axis=1)

y_pred = AveragingBlending(models, x_train, y_train, x_test)
y_pred
## log 값 취한것 다시 변환 해준다

#y_pred = np.expm1(y_pred)
#y_pred

array([ 537962.41482426,  455676.26599676, 1332045.56818833, ...,
        454855.63094488,  285156.17709279,  436964.63828615])

#### 2. 하이퍼파라미터 튜닝




## 4. 모델평가

## 5. 저장 및 제출

In [ ]:
data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'

submission_path = join(data_dir, 'sample_submission.csv')
submission = pd.read_csv(submission_path)
submission.head()

In [127]:
result = pd.DataFrame({
    'id' : sub_id, 
    'price' : y_pred
})

result.head()

,id,price
0,15035,5.379624e+05
1,15036,4.556763e+05
2,15037,1.332046e+06
3,15038,3.267744e+05
4,15039,3.172921e+05


In [132]:
data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'

submission_path = join(data_dir, 'sample_submission.csv')
submission = pd.read_csv(submission_path)
submission.head()

## 값 넣기

submission['price'] = y_pred
submission.head()

#  csv 파일로 저장

submission_csv_path = '{}/submission_{}_RMSLE_{}.csv'.format(data_dir, 'lgbm', '0.164399')
submission.to_csv(submission_csv_path, index=False)
print(submission_csv_path)

/aiffel/aiffel/kaggle_kakr_housing/data/submission_lgbm_RMSLE_0.164399.csv
